In [12]:
import os
import nltk

# https://stackoverflow.com/questions/51390676/how-to-visualize-pyspark-mls-lda-or-other-clustering

nltk.download('stopwords')
from nltk.corpus import stopwords

from pyspark import SparkConf, SparkContext,SQLContext
from pyspark.sql import SparkSession, functions
from pyspark.ml.feature import Word2Vec,CountVectorizer,Tokenizer, StopWordsRemover
from pyspark.ml.clustering import LDA, LDAModel
from pyspark.sql.functions import col, udf, countDistinct, regexp_replace
from pyspark.sql.types import IntegerType,ArrayType,StringType
import pandas as pd
import numpy as np
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import lit
import csv

def ith_(v, i):
    try:
        return float(v[i])
    except ValueError:
        return None

spark = SparkSession \
.builder \
.appName("Python Spark SQL basic example") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()

months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
#months = ["January"]
years = ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]
#years = ["2010"]
#lookup = {"January": "01", "February":"02", "March":"03", "April":"04", "May":"05", "June": "06", "July":"07", "August":"08", "September":"09", "October":"10", "November":"11", "December":"12"}
with open("../processed_data/lines/data.csv", 'w') as file:
    header = ["date", "score", "comments", "num_posts"]
    writer = csv.writer(file)
    writer.writerow(header)
    for year in years:
        for month in months:
            csv_path = "/" + year + "/" + month + ".csv"
            json_path = "/" + year + "/" + month + ".json"
            data_path = "../data" + csv_path # Data path for csv file
            spark_df = spark.read.csv(data_path, inferSchema = True, header=True) # checking the csv file
            score = int(spark_df.agg({'Score': 'sum'}).first()[0])
            com = int(spark_df.agg({'Num Comments': 'sum'}).first()[0])
            data = [year + "-" + lookup[month], score, com, spark_df.count()]
            writer.writerow(data)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alirahman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
